In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM


import torch.onnx
import torch
import onnx
import numpy as np
import onnxruntime as ort
from workload_generator import parse_onnx, write_csv



/home/jeffjw/anaconda3/envs/DWG/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

# Load the pretrained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("TheBloke/law-LLM-AWQ")
model = AutoModelForCausalLM.from_pretrained("TheBloke/law-LLM-AWQ")

# Attach hook
def store_input_hook(module, input, output):
    module.input_hooked = input

hooked = {}
# Do a recursive search for all modules that are of type "lookup_modules" and register the hook
def register_hook(module, hook, lookup_modules = [torch.nn.Linear]):
    for child in module.children():
        if isinstance(child, tuple(lookup_modules)):
            handle = child.register_forward_hook(store_input_hook)
            hooked[child] = handle
        else:
            register_hook(child, store_input_hook, lookup_modules)

register_hook(model, hooked, lookup_modules = [torch.nn.Linear, type(model.transformer.h[0].mlp.c_fc)])

print(hooked)


# Example usage
input_text = "Hello, how are you?"
input_ids = tokenizer.encode(input_text, return_tensors="pt")
output = model.generate(input_ids)
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

print(decoded_output)


# Set the path for saving the ONNX file
onnx_file_path = "model.onnx"

# Set the input tensor
input_ids = torch.tensor([[15496, 11, 703, 389, 345, 30]])

# Export the model to ONNX
torch.onnx.export(model, input_ids, onnx_file_path)


In [4]:
tokenizer = AutoTokenizer.from_pretrained("GPT2-XL")
model = AutoModelForCausalLM.from_pretrained("GPT2-XL")

In [ ]:
keys = [k.input_hooked[0] for k in hooked.keys()]

In [4]:
in_A = keys[0][0,0]
in_B = list(hooked.keys())[0].weight[:,:4] * 16

NameError: name 'keys' is not defined

In [ ]:
ref_out = in_A @ in_B

NameError: name 'in_A' is not defined

In [ ]:
# Load the ONNX model
onnx_model = onnx.load("model.onnx")


In [ ]:
# Parse the ONNX model
parsed_model, other_ops = parse_onnx(onnx_model)

In [ ]:
pdict = {k: v for k,v in parsed_model.items() if v.op_type == 'Gemm' or v.op_type == 'MatMul'}
pdict['transformer.h.1.attn.MatMul']

In [ ]:
# Write the parsed model to a CSV file
write_csv(parsed_model, "gpt2_gen.csv")

In [ ]:
import torch 
